In [1]:
import MetaTrader5 as mt5
from utils.mt5_utils import create_dataframe

mt5.initialize()
df = create_dataframe(currency_pair='eurusd', timeframe="d1", years_back=3)

df.tail()

,open,high,low,close,tick_volume,spread
time,,,,,,
2024-08-30,1.10800,1.10945,1.10434,1.10474,42223,8
2024-09-01,1.10468,1.10494,1.10437,1.10442,1527,8
2024-09-02,1.10443,1.10769,1.10414,1.10670,25779,8
2024-09-03,1.10670,1.10697,1.10259,1.10480,43184,7
2024-09-04,1.10480,1.10949,1.10396,1.10659,34681,7


In [2]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(df):
    adf_result = adfuller(df)
    if(adf_result[1] < 0.05):
        return True
    else:
        return False

In [3]:
check_stationarity(df['close'])

False

In [4]:
check_stationarity(df['close'].diff().dropna())

True

In [5]:
split = int(len(df)*0.7)

data_train = df[:split]
data_test = df[split:]

# df.shape[0] # 1095, 
data_train.shape[0], data_test.shape[0]

(766, 329)

In [6]:
import itertools
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Ignore specific warnings
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.filterwarnings("ignore", message="Non-stationary starting autoregressive parameters found")
warnings.filterwarnings("ignore", message="Non-invertible starting MA parameters found")


def get_best_params(data):
    # Define the p, d and q parameters
    p = q = range(0, 5)
    d = range(1, 2)

    # Generate all different combinations of p, d and q triplets
    pdq = list(itertools.product(p, d, q))

    best_aic = float("inf")
    best_param = None

    for param in pdq:
        try:
            model = ARIMA(data.close.values, order=param)
            model_fit = model.fit()
            if model_fit.aic < best_aic:
                best_param = param
                best_aic = model_fit.aic
        except:
            continue
    return best_param

best_param = get_best_params(data_train)
print("The best arima params: ", best_param)

The best arima params:  (0, 1, 0)


In [7]:
def get_predicted_prices(close_prices, best_param):
    # Define the best model
    best_model = ARIMA(close_prices.values, order=best_param)
    best_model_fit = best_model.fit(method_kwargs={"warn_convergence": False})
    # Make forecast
    predictions = best_model_fit.forecast(steps=1)[0]
    return predictions

In [8]:
import numpy as np

# Ensure data_train is a copy of the original DataFrame to avoid SettingWithCopyWarning
data_train = df[:split].copy()

# Define the window size
window = 700

# Initialize the 'predicted_price' column with NaNs
data_train['predicted_price'] = np.nan

# Use .loc[] to set values to avoid SettingWithCopyWarning and ChainedAssignmentError
for i in range(window, data_train.shape[0]):
    # Ensure we are using .loc to avoid the warnings
    data_train.loc[data_train.index[i], 'predicted_price'] = get_predicted_prices(
        data_train.iloc[i-window:i].close, best_param
    )


In [12]:
# ..........same dates for both...................
# data_train['predicted_price'].tail()
# df['close'].loc['2023-08-10':'2023-08-15']
# .........diffent dates for actual...............
# df['close'].loc['2023-08-09':'2023-08-14']

""" 
time          predicted  actual  |               actual
2023-08-10    1.09768    1.09812 | 2023-08-09    1.09768
2023-08-11    1.09812    1.09468 | 2023-08-10    1.09812
2023-08-13    1.09468    1.09428 | 2023-08-11    1.09468
2023-08-14    1.09428    1.09073 | 2023-08-13    1.09428
2023-08-15    1.09073    1.09034 | 2023-08-14    1.09073
"""
data_train['predicted_price'].tail()

time
2023-08-10    1.09768
2023-08-11    1.09812
2023-08-13    1.09468
2023-08-14    1.09428
2023-08-15    1.09073
Name: predicted_price, dtype: float64

In [13]:
df['close'].loc['2023-08-10':'2023-08-15']

time
2023-08-10    1.09812
2023-08-11    1.09468
2023-08-13    1.09428
2023-08-14    1.09073
2023-08-15    1.09034
Name: close, dtype: float64

In [11]:
# https://www.kaggle.com/code/luigimersico/forecasting-foreign-exchange-rate-with-arima